In [1]:
import matplotlib.pyplot as plt
%config Completer.use_jedi = False
import tqdm
%config InlineBackend.figure_format = 'retina'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
#mpl.rcParams['figure.figsize'] = (6, 4)
import numpy as np
import h5py

%load_ext autoreload
%autoreload 2

# Poisson Binomial playground

In [65]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Slider, CustomJS, Paragraph
from bokeh.layouts import column, row
from scipy import stats


output_file('../interactive/poissonbinomial.html')

n = 10
data = {'left_edges':np.arange(-0.5,n),
        'right_edges':1.0+np.arange(-0.5,n),
        'values':stats.binom(n=n,p=0.5).pmf(np.arange(n+1))}
        #'binomial':stats.binom(n=n,p=0.5).pmf(np.arange(n+1))}

source = ColumnDataSource(data=data)
p = figure(title='Poisson Binomial',plot_width=500, plot_height=600,
           background_fill_color="#fafafa",tools='')
p.x_range.start = -0.5
p.x_range.end = n+0.5
p.y_range.start = 0
p.xaxis.ticker = [i for i in range(n+1)]
#p.x_range.end = tend
p.xaxis.axis_label = 'Number of heads'
p.yaxis.axis_label = 'Probability mass'
p.xgrid.visible = False

#p.line('t', 'existing', line_width=1,source=source)
p.quad(top='values', bottom=0, left='left_edges', right='right_edges', line_color="white", alpha=1.0,source=source)
#p.quad(top='binomial', bottom=0, left='left_edges', right='right_edges', line_color="orange", fill_alpha=0.0, alpha=1.0,source=source)

# Add a slider to control which year the patches are coloured by
probability_sliders = [Slider(start=0, end=1, value=0.5, step=0.01, title="Probability "+str(i+1), orientation="horizontal") for i in range(n)]

# Add the custom javascript to interface the slider with the patches
callback = CustomJS(args={**{'source':source}, **{f'probability_{i}':probability_sliders[i] for i in range(n)}}, code=f"""

        function poisson_binomial(probs, probslen, result) {{
            var i;
            var j;
            var oldlen = 2; // length of old kernel
            var signal = [0,0];
            var t;
            var tmp;

            // initialize (old kernel)
            result[0] = 1-probs[0];
            result[1] = probs[0];

            // loop through all other probs
            for (i=1; i < probslen; i++){{

                // set signal
                signal[0] = probs[i];
                signal[1] = 1-probs[i];

                // initialize result and calculate the two edge cases
                result[oldlen] = signal[0] * result[oldlen-1];

                t = result[0];
                result[0] = signal[1]*t;

                //calculate the interior cases
                for(j=1; j < oldlen; j++){{
                    tmp=result[j];
                    result[j] = signal[0] * t + signal[1] * result[j];
                    t=tmp;
                }}

            oldlen++;
            }}
        }}
        
        const p = probability_0.value;
        const data = source.data;
        const x = data['values']
        const n = {n};
        var probs = [{','.join(['probability_'+str(i)+'.value' for i in range(n)])}]
        poisson_binomial(probs,n,x);
        source.change.emit();
    """)
for i in range(n):
    probability_sliders[i].js_on_change('value', callback)
    
para = Paragraph(text=f"""You are flipping {n} weighted coins. You can set the probability that each coin shows heads below. Given those probabilities, the histogram shows the probability of each number of heads.""",
width=200, height=100)

layout = column(row(p,column([para]+probability_sliders)))
show(layout)

# Beta playground

In [2]:
with h5py.File('./data/gaiaedr3_grid.h5', 'r') as f:
    grid = f['grid'][:]
    extent = f['extent'][:]

In [82]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Slider, CustomJS, Paragraph, Range1d, PointDrawTool, HoverTool
from bokeh.layouts import column, row
from bokeh.models.mappers import LinearColorMapper, LogColorMapper
from bokeh.events import DoubleTap
from scipy import stats
n_init, k_init, color_init = [50,40,270],[40,15,35],['red','blue','green']
N_init = len(n_init)

TOOLTIPS = [
    ("(n,k)", "($x{0,0}, $y{0,0})"),
    ("# of stars", "@image{0,0}"),
]
X = grid.sum(axis=0).T[:301,:301].astype(float)
for n in range(301):
    for k in range(n):
        if X[n,k] < 0.5:
            X[n,k] = np.nan
X[:5] = np.nan
p1_source = ColumnDataSource(data={'image':[X]})


p1 = figure(title='Observations and measurements of stars in Gaia EDR3',plot_width=500, plot_height=500,
           background_fill_color="#fafafa",tools='pan,box_zoom,reset', x_range=Range1d(0,300,bounds=(0, 300)), y_range=Range1d(0,300,bounds=(0, 300)))

p1.xgrid.visible = False
p1.ygrid.visible = False
p1.xaxis.axis_label = 'Predicted number of observations'
p1.yaxis.axis_label = 'Number of astrometric measurements'
p1_image = p1.image(image='image', x=0, y=0, dw=300, dh=300,level="image", color_mapper=LogColorMapper(palette="Spectral11",low=1), source=p1_source)
p1_hover = HoverTool(renderers=[p1_image],
                         tooltips=TOOLTIPS)
p1.add_tools(p1_hover)
p1.line([0,300], [0,300], line_width=1, line_color = 'white', line_dash='dashed')

p1_line_source = ColumnDataSource(data={'x':n_init,'y':k_init,'color':color_init})
c1 = p1.circle('x', 'y', size=10,source=p1_line_source, color='color')
draw_tool = PointDrawTool(renderers=[c1],add=False)
p1.add_tools(draw_tool)

p2 = figure(title='Posterior on the measurement probability',plot_width=500, plot_height=500,
           background_fill_color="#fafafa",tools='',x_range=Range1d(0,1,bounds=(0, 1)))
p2.y_range.start = 0
p2.xaxis.axis_label = 'Measurement probability'
p2.yaxis.axis_label = 'Probability density'

c = 5

data = {}
#data['X'] = np.linspace(1e-3,1-1e-3,1001)
data['x'] = [np.linspace(1e-3,1-1e-3,1001)]*3
data['y'] = [stats.beta(a=1+k_init[i],b=1+n_init[i]-k_init[i]).pdf(data['x'][i]) for i in range(N_init)]
data['color'] = color_init

p2_source = ColumnDataSource(data=data)
p2.multi_line('x', 'y', line_width=1,source=p2_source,line_color='color')

# Add the custom javascript to interface the slider with the patches
callback = CustomJS(args=dict(p1_source=p1_line_source, p2_source=p2_source), code="""

        
        function lnBetaFunc(a, b) {
            // Log Beta Function
            // ln(Beta(x,y))
            let foo = 0.0;

            for (var i=0; i<a-2; i++) {
                foo += Math.log(a-1-i);
            }
            for (var i=0; i<b-2; i++) {
                foo += Math.log(b-1-i);
            }
            for (var i=0; i<a+b-2; i++) {
                foo -= Math.log(a+b-1-i);
            }
            return foo;
        }
        function betaFunc(x,y) {
            // Beta Function
            // Beta(x,y) = e^(ln(Beta(x,y))
            return Math.exp(lnBetaFunc(x,y));
        }
        function lnBetaPDF(x, a, b) {
            // Log of the Beta Probability Density Function
            return ((a-1)*Math.log(x) + (b-1)*Math.log(1-x)) - lnBetaFunc(a,b)
        }
        function betaPDF(x, a, b) {
            // Beta probability density function impementation
            // using logarithms, no factorials involved.
            // Overcomes the problem with large integers
            return Math.exp(lnBetaPDF(x, a, b))
        }
        
        var p1_data = p1_source.data;
        var p2_data = p2_source.data;
        
        // Loop over dots
        for (var i = 0; i < 3; i++) {
            let n = Math.floor(p1_data['x'][i]);
            let k = Math.floor(p1_data['y'][i]);
            let new_alpha = 1 + k;
            let new_beta = 1 + n - k;
            let x = p2_data['x'][i]
            let y = p2_data['y'][i]
            if (k <= n) {
                var lnB = lnBetaFunc(new_alpha, new_beta);
                for (var j = 0; j < x.length; j++) {
                    y[j] = Math.exp( (new_alpha-1)*Math.log(x[j]) + (new_beta-1)*Math.log(1-x[j])-lnB);
                }
            }
        }
        p2_source.change.emit();
    """)
p1_line_source.js_on_change("data", callback)
para = Paragraph(text=f"""The left panel shows a 2D histogram of stars in Gaia EDR3 by their predicted number of observations and actual number of astrometric measurements. 
You can hover to see the number of stars in each cell.
Each circle with centre (n, k) corresponds to one of the lines in the right panel. If Gaia was to observe a star n times and obtain k measurements, then the line shows your posterior on the measurement probability, assuming that every observation was equally likely to result in a measurement.
If you enable the 'Point Draw Tool' on the toolbar then you can click-and-drag the three circles and see the posteriors update.
Note that we have not corrected these posteriors for the selection effect due to stars only appearing in Gaia EDR3 if they have at least five astrometric measurements.""",
width=200, height=100)

show(column(row(p1,p2),para))

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import special
import healpy as hp

# Load in parameters
#directory = '/mnt/extraspace/GaiaSelectionFunction/Code/C++/Test/SlimTest/'
directory = '/Users/douglasboubert/Science/gaia-selection-function/Gaia-EDR3-selection-function/Python/data/DataSubsetRun/'
params = pd.read_csv(directory+'Optimiser_Properties.dat',skipinitialspace=True)
Nt = int(params['Nt'][0])
Nm = int(params['Nm'][0])
Nl = int(params['Nl'][0])

# Load in gaps
gaps = pd.read_csv('/Users/douglasboubert/Science/gaia-selection-function/Gaia-EDR3-selection-function/Python/data/edr3_gaps.csv')

# Load in transformed parameters
file = 'TempPositions/TempPosition_TransformedParameters.dat'
xt = pd.read_csv(directory+file,header=None)[0][:Nt].values

In [ ]:
numpy.loadtxt(file,max_rows=Nt)
        source.data[str(n)] = special.expit(pd.read_csv(file,header=None,nrows=Nt)[0].values)

In [16]:
%time np.loadtxt(directory+file,max_rows=Nt)

CPU times: user 255 ms, sys: 6.9 ms, total: 262 ms
Wall time: 262 ms


array([-3.33821, -3.83329, -4.1871 , ...,  2.90821,  3.3169 ,  3.28357])

In [17]:
%time pd.read_csv(directory+file,header=None,nrows=Nt)[0].values

CPU times: user 12.4 ms, sys: 5.02 ms, total: 17.4 ms
Wall time: 15.8 ms


array([-3.33821, -3.83329, -4.1871 , ...,  2.90821,  3.3169 ,  3.28357])

In [3]:
tbeg, tend = 1717.6256+(np.linspace(1666.4384902198801, 2704.3655735533684, 2) + 2455197.5 - 2457023.5 - 0.25)*4
t = np.linspace(tbeg,tend,Nt+1)
t = 0.5*(t[1:]+t[:-1])



In [6]:
from bokeh.io import show
from bokeh.models import CustomJS, Select

select = Select(title="Option:", value="foo", options=["foo", "bar", "baz", "quux"])
select.js_on_change("value", CustomJS(code="""
    console.log('select: value=' + this.value, this.toString())
"""))

show(select)

In [7]:
select.value

'foo'

In [18]:
# Load in parameters
#directory = '/mnt/extraspace/GaiaSelectionFunction/Code/C++/Test/SlimTest/'
directory = './data/SmoothingTest/'
params = pd.read_csv(directory+'Optimiser_Properties.dat',skipinitialspace=True)
Nt = int(params['Nt'][0])
Nm = int(params['Nm'][0])
Nl = int(params['Nl'][0])

tbeg, tend = 1717.6256+(np.linspace(1666.4384902198801, 2704.3655735533684, 2) + 2455197.5 - 2457023.5 - 0.25)*4
t = np.linspace(tbeg,tend,Nt+1)
t = 0.5*(t[1:]+t[:-1])
data = {'t':t}

# Load in transformed parameters
import glob
files = glob.glob(directory+'TempPositions/TempPosition*_TransformedParameters.dat')
#file = 'TempPositions/TempPosition_TransformedParameters.dat'

N = 0
for file in files:
    epoch = file.split('/')[-1].split('_')[0][12:]
    data[epoch] = special.expit(pd.read_csv(file,header=None)[0][:Nt].values)
    N += 1
data['existing'] = data['1']

In [19]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Slider, CustomJS, Button
from bokeh.layouts import column, row

data = {'t':t,'1':special.expit(xt)}
N = 100
for i in range(2,N+1):
    data[str(i)] = special.expit(xt+np.random.normal(0,0.1,xt.size))
data['existing'] = data['1']

source = ColumnDataSource(data=data)
output_file('pt.html')
p = figure(title='Detection probability with time',plot_width=1200, plot_height=400,
           background_fill_color="#fafafa",tools='reset,xbox_zoom,xpan')
p.y_range.start = 0
p.y_range.end = 1
p.x_range.start = tbeg
p.x_range.end = tend
p.xaxis.axis_label = 'OBMT (revolutions)'
p.yaxis.axis_label = 'Detection probability (pt)'

p.line('t', 'existing', line_width=1,source=source)

# Add a slider to control which year the patches are coloured by
epoch_slider = Slider(start=1, end=N, value=1, step=1, title="Epoch", orientation="horizontal", css_classes=["custom-slider"])

# Add the custom javascript to interface the slider with the patches
callback = CustomJS(args=dict(source=source, epoch=epoch_slider), code="""
        source.data['existing'] = source.data[String(epoch.value)]
        source.change.emit();
    """)
epoch_slider.js_on_change('value', callback)
epoch_slider.end = 3

def do_stuff():

    pass

refresh_button = Button(label='Refresh')
refresh_button.on_click(do_stuff)

# Combine plot and slider and output
layout = row(p,column(epoch_slider,refresh_button))
show(layout)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [22]:
dict(source.data)

{'t': array([1078.40039421, 1078.44206088, 1078.48372755, ..., 5229.98372755,
        5230.02539421, 5230.06706088]),
 '1': array([0.03428337, 0.02118001, 0.01496298, ..., 0.9482508 , 0.96500405,
        0.96386084]),
 '2': array([0.02902907, 0.02184784, 0.01335512, ..., 0.94558341, 0.96345837,
        0.96455747]),
 '3': array([0.03253576, 0.02022666, 0.01483734, ..., 0.94846173, 0.96446481,
        0.9587469 ]),
 '4': array([0.0401713 , 0.02179278, 0.01801849, ..., 0.94215454, 0.96407669,
        0.96466903]),
 '5': array([0.03458377, 0.02157403, 0.01636152, ..., 0.94522904, 0.96267337,
        0.96185821]),
 '6': array([0.02831198, 0.02487712, 0.01520487, ..., 0.95414195, 0.96184913,
        0.96515049]),
 '7': array([0.03610651, 0.02251496, 0.01454108, ..., 0.94597526, 0.96158279,
        0.96450597]),
 '8': array([0.03497569, 0.02015811, 0.01410627, ..., 0.94527793, 0.96917257,
        0.96710448]),
 '9': array([0.03987579, 0.02205058, 0.01347089, ..., 0.95131084, 0.96687625,
    

In [15]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Slider, CustomJS
from bokeh.layouts import column, row

data = {'x':np.linspace(1e-3,1-1e-3,1000),'y':0.5*np.ones(1000)}
source = ColumnDataSource(data=data)

output_file('coinflipping.html')
p = figure(title='Detection probability with time',plot_width=600, plot_height=600,
           background_fill_color="#fafafa")
p.y_range.start = 0.0
p.x_range.start = 0
p.x_range.end = 1
p.xaxis.axis_label = 'OBMT (revolutions)'
p.yaxis.axis_label = 'Detection probability (pt)'

p.line('x', 'y', line_width=2,source=source)

# Add a slider to control which year the patches are coloured by
alpha_slider = Slider(start=-2, end=2, value=0.0, step=0.001, title="log10(a)")
beta_slider = Slider(start=-2, end=2, value=0.0, step=0.001, title="log10(b)")

# Add the custom javascript to interface the slider with the patches
callback = CustomJS(args=dict(source=source, alpha=alpha_slider, beta=beta_slider), code="""

        var g = 7;
        var C = [0.99999999999980993, 676.5203681218851, -1259.1392167224028,771.32342877765313, -176.61502916214059, 12.507343278686905, -0.13857109526572012, 9.9843695780195716e-6, 1.5056327351493116e-7];

        function gamma(z) {

            if (z < 0.5) return Math.PI / (Math.sin(Math.PI * z) * gamma(1 - z));
            else {
                z -= 1;

                var x = C[0];
                for (var i = 1; i < g + 2; i++)
                x += C[i] / (z + i);

                var t = z + g + 0.5;
                return Math.sqrt(2 * Math.PI) * Math.pow(t, (z + 0.5)) * Math.exp(-t) * x;
            }
        }
        
        var data = source.data;
        const new_alpha = Math.pow(10,alpha.value);
        const new_beta = Math.pow(10,beta.value);
        const x = data['x']
        const y = data['y']
        var B = gamma(new_alpha)*gamma(new_beta)/gamma(new_alpha+new_beta);
        for (var i = 0; i < x.length; i++) {
            y[i] = Math.pow(x[i],new_alpha-1.0)*Math.pow(1.0-x[i],new_beta-1.0)/B;
        }
    
        source.change.emit();
    """)
alpha_slider.js_on_change('value', callback)
beta_slider.js_on_change('value', callback)

# Combine plot and slider and output
layout = column(p,alpha_slider, beta_slider)
show(layout)